In [2]:
# her we have the core libraries
import numpy as np
import pandas as pd
import os
import cv2
from collections import Counter
# and her the visulization one
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.axes_grid1 import ImageGrid

# her we have the machine learning and preprocessing things
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, classification_report, confusion_matrix, multilabel_confusion_matrix
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.pipeline import Pipeline
# the tensorflow and keras for the CNN
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.optimizers import Adam, SGD , Nadam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import EfficientNetB0, ResNet50, DenseNet121, VGG16, InceptionV3
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical

#imahe processing
from PIL import Image
import albumentations as A
# utility Libraries
import time
import logging
import gc
# system Librarie
import sys

In [5]:

full_df_file = "eye_disease_detection_dataset/archive/full_df.csv"
df = pd.read_csv(full_df_file)
df.head()

,ID,Patient Age,Patient Sex,Left-Fundus,Right-Fundus,Left-Diagnostic Keywords,Right-Diagnostic Keywords,N,D,G,C,A,H,M,O,filepath,labels,target,filename
0,0,69,Female,0_left.jpg,0_right.jpg,cataract,normal fundus,0,0,0,1,0,0,0,0,../input/ocular-disease-recognition-odir5k/ODI...,['N'],"[1, 0, 0, 0, 0, 0, 0, 0]",0_right.jpg
1,1,57,Male,1_left.jpg,1_right.jpg,normal fundus,normal fundus,1,0,0,0,0,0,0,0,../input/ocular-disease-recognition-odir5k/ODI...,['N'],"[1, 0, 0, 0, 0, 0, 0, 0]",1_right.jpg
2,2,42,Male,2_left.jpg,2_right.jpg,laser spot，moderate non proliferative retinopathy,moderate non proliferative retinopathy,0,1,0,0,0,0,0,1,../input/ocular-disease-recognition-odir5k/ODI...,['D'],"[0, 1, 0, 0, 0, 0, 0, 0]",2_right.jpg
3,4,53,Male,4_left.jpg,4_right.jpg,macular epiretinal membrane,mild nonproliferative retinopathy,0,1,0,0,0,0,0,1,../input/ocular-disease-recognition-odir5k/ODI...,['D'],"[0, 1, 0, 0, 0, 0, 0, 0]",4_right.jpg
4,5,50,Female,5_left.jpg,5_right.jpg,moderate non proliferative retinopathy,moderate non proliferative retinopathy,0,1,0,0,0,0,0,0,../input/ocular-disease-recognition-odir5k/ODI...,['D'],"[0, 1, 0, 0, 0, 0, 0, 0]",5_right.jpg


In [21]:
nested_odir_path = 'eye_disease_detection_dataset/archive/ODIR-5K/ODIR-5K/'

main_files = os.listdir(nested_odir_path)
print("Main ODIR-5K folder contents:")
print(main_files)

# List the contents of the testing_images and training_images folders
testing_images_path = os.path.join(nested_odir_path, 'Testing Images')
training_images_path = os.path.join(nested_odir_path, 'Training Images')

print("\nTesting images folder contents:")
print(os.listdir(testing_images_path)[:10])  # Display first 10 files/directories

print("\nTraining images folder contents:")
print(os.listdir(training_images_path)[:10])

Main ODIR-5K folder contents:
['data.xlsx', 'Testing Images', 'Training Images']

Testing images folder contents:
['1000_left.jpg', '1000_right.jpg', '1001_left.jpg', '1001_right.jpg', '1002_left.jpg', '1002_right.jpg', '1003_left.jpg', '1003_right.jpg', '1004_left.jpg', '1004_right.jpg']

Training images folder contents:
['0_left.jpg', '0_right.jpg', '1005_left.jpg', '1005_right.jpg', '1006_left.jpg', '1006_right.jpg', '1008_left.jpg', '1008_right.jpg', '100_left.jpg', '100_right.jpg']


In [ ]:
preprocessed_path = "eye_disease_detection_dataset/archive/preprocessed_images/"
preprocessed_files = os.listdir(preprocessed_path)
print(f"Number of files in preprocessed_images folder: {len(preprocessed_files)}")

# Display first few files
print("Sample files from preprocessed_images folder:")
print(preprocessed_files[:10])